In [1]:
%run "Setup_DesignStart.ipynb"

Serial baud rate = 38400


In [2]:
print(target.get_target_name())
print(target.get_fw_buildtime())
print(target.get_fpga_buildtime())

ArmTrace
DesignStart Cortex M3, compiled Jun 29 2020, 15:06:00
None
FPGA build time: 7/29/2020, 15:21


In [15]:
target.fpga_write(target.REG_SOFT_TRIG_ENABLE, [1])
target.fpga_write(target.REG_COUNTER_QUICK_START, [1])
target.fpga_write(target.REG_CAPTURE_RAW, [0])
target.fpga_write(target.REG_COUNT_WRITES, [0])
target.fpga_write(target.REG_SOFT_TRIG_PASSTHRU, [1])

In [16]:
# match on any PC trace packet:
target.set_pattern_match(0, [5, 8, 32, 0, 0, 0, 0, 0], [255, 255, 255, 0, 0, 0, 0, 0])
#target.set_pattern_match(0, [5, 8, 32, 229, 2, 0, 0, 132], [255, 255, 255, 255, 255, 255, 255, 0])

# enable matching rule:
target.fpga_write(target.REG_PATTERN_ENABLE, [1])

target.fpga_write(target.REG_CAPTURE_LEN, int.to_bytes(10000, length=4, byteorder='little'))

In [17]:
target.set_reg('DWT_COMP0', '000012a4')
target.set_reg('DWT_COMP1', '000012fc')

In [18]:
target.set_reg('ETM_TEEVR', '000150a0') # COMP0 or COMP1
#target.set_reg('ETM_TEEVR', '00000020') # COMP0 only
#target.set_reg('ETM_TEEVR', '00000021') # COMP1 only

In [19]:
target.arm_trace()

In [20]:
target.synced()

In [21]:
target.simpleserial_write('p', bytearray(16), printresult=True)

r7DF76B0C1AB899B33E42F047B91B546F


In [ ]:
#Capture Traces
from tqdm import tnrange
import numpy as np
import time

num_traces = 5
ktp = cw.ktp.Basic()

traces = []

for i in tnrange(num_traces, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    trace = cw.capture_trace(scope, target.ss, text, key)

    if trace is None:
        continue
    traces.append(trace)

#Convert traces to numpy arrays
trace_array = np.asarray([trace.wave for trace in traces])  # if you prefer to work with numpy array for number crunching
textin_array = np.asarray([trace.textin for trace in traces])
known_keys = np.asarray([trace.key for trace in traces])  # for fixed key, these keys are all the same

In [22]:
target.fpga_read(target.REG_TRACE_COUNT, 8)

array('B', [0, 0, 0, 0, 0, 0, 0, 144])

In [23]:
target.fifo_empty()

False

In [24]:
raw = target.read_capture_data()

In [25]:
len(raw)

26

In [26]:
#target.print_raw_data(raw)

In [27]:
times = target.get_rule_match_times(raw, rawtimes=False, verbose=True)

     158 rule # 0, delta = 158, adjust = 6
     378 rule # 0, delta = 220, adjust = 6
     662 rule # 0, delta = 284, adjust = 6
     886 rule # 0, delta = 224, adjust = 6
    1170 rule # 0, delta = 284, adjust = 6
    1394 rule # 0, delta = 224, adjust = 6
    1678 rule # 0, delta = 284, adjust = 6
    1902 rule # 0, delta = 224, adjust = 6
    2186 rule # 0, delta = 284, adjust = 6
    2410 rule # 0, delta = 224, adjust = 6
    2694 rule # 0, delta = 284, adjust = 6
    2918 rule # 0, delta = 224, adjust = 6
    3202 rule # 0, delta = 284, adjust = 6
    3426 rule # 0, delta = 224, adjust = 6
    3714 rule # 0, delta = 288, adjust = 6
    3934 rule # 0, delta = 220, adjust = 6
    4218 rule # 0, delta = 284, adjust = 6
    4442 rule # 0, delta = 224, adjust = 6


In [ ]:
times = target.get_rule_match_times(raw, rawtimes=True, verbose=True)

In [ ]:
len(times)

In [ ]:
times = target.get_rule_match_times(raw, True)

In [ ]:
times

In [ ]:
# plot!

In [ ]:
if scope.clock.adc_src == 'clkgen_x4' or scope.clock.adc_src == 'extclk_x4':
    multiplier = 4
else:
    multiplier = 1

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.resources import INLINE
from bokeh.models import Span

output_notebook(INLINE)
p = figure(plot_width=2000)

xrange = range(len(traces[0].wave))
p.line(xrange, traces[0].wave, line_color="red")

vlines = []
for t in times:
    vlines.append(Span(location=t[0]*multiplier, dimension='height', line_color='black', line_width=1))

p.renderers.extend(vlines)

show(p)

In [ ]:
def update_plot(trace=0, markers=1):

    r.data_source.data['y'] = traces[trace].wave
    push_notebook()

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, push_notebook
from bokeh.resources import INLINE
from bokeh.models import Span
from ipywidgets import interact, Layout

output_notebook(INLINE)
p = figure(plot_width=2000)

xrange = range(len(traces[0].wave))
r = p.line(xrange, traces[0].wave, line_color="red")

vlines = []
for t in times:
    vlines.append(Span(location=t[0]*multiplier, dimension='height', line_color='black', line_width=1))

p.renderers.extend(vlines)

In [ ]:
show(p, notebook_handle=True)

In [ ]:
interact(update_plot, trace=(0, len(traces)-1), markers=(0, 1))